In [ ]:
using Plots
using LinearAlgebra
using Printf
using LaTeXStrings
using DifferentialEquations

In [ ]:
default(lw=2, markersize=6,
    xtickfont=font(12), ytickfont=font(12),
    guidefont=font(14), legendfont=font(12), titlefont=font(12))

Solve the heat equation
$$
u_t = \kappa u_{xx},
$$
with periodic boundary conditions using finite differences.

In [ ]:
L = 10;
κ = 1.0;

N = 50;
x = LinRange(0, L, N + 1)[1:end-1];
Δx = x[2] - x[1];
@show Δt = 0.1 * Δx^2;
n_steps = 5000;

u0 = @. exp(-(x - L / 2)^2);
plot(x, u0, label="")
xlabel!(L"$x$")
ylabel!(L"$u_0$")

In [ ]:
function integrate_heat_equation1(u0, κ, Δx, Δt, n_steps)
    u = deepcopy(u0)
    t = 0.0

    u_traj = [deepcopy(u)]
    t_traj = Float64[t]

    u_new = similar(u)

    N = length(u)

    for n in 1:n_steps
        # update u with periodic boundary conditions
        u_new[1] = u[1] + κ * Δt / Δx^2 * (u[2] - 2 * u[1] + u[end])
        for j in 2:N-1
            u_new[j] = u[j] + κ * Δt / Δx^2 * (u[j+1]- 2*u[j] + u[j-1])
        end
        u_new[end] = u[end] + κ * Δt / Δx^2 * (u[1] - 2 * u[end] + u[end-1])
        # copy over
        @. u = u_new

        t += Δt

        push!(u_traj, deepcopy(u))
        push!(t_traj, t)
    end

    return t_traj, u_traj

end

In [ ]:
t_traj, u_traj = integrate_heat_equation1(u0, κ, Δx, Δt, n_steps);

In [ ]:
anim = @animate for (n, t) in enumerate(t_traj[1:100:end])
    plot(x, u_traj[n], label="")
    title!(@sprintf("t = %.2f", t))
    xlabel!(L"$x$")
    ylims!(0,1.5)
end

In [ ]:
gif(anim, fps=15)


In [ ]:
function fheat!(du, u, p, t)
    κ = p[1]
    Δx = p[2]
    N = length(u)

    du[1] = κ / Δx^2 * (u[2] - 2 * u[1] + u[end])
    for j in 2:N-1
        du[j] = κ / Δx^2 * (u[j+1] - 2 * u[j] + u[j-1])
    end
    du[end] =κ / Δx^2 * (u[1] - 2 * u[end] + u[end-1])

    du
end

In [ ]:
p = [κ, Δx]
tspan = (0, 10);
prob = ODEProblem(fheat!, u0, tspan, p);


In [ ]:
t_vals = 0:0.1:10;

sol = solve(prob, saveat=t_vals);

In [ ]:
anim = @animate for (t, u) in zip(sol.t, sol.u)
    plot(x, u, label="")
    title!(@sprintf("t = %.2f", t))
    xlabel!(L"$x$")
    ylims!(0, 1.5)
end

In [ ]:
gif(anim, fps=15)
